In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
from ffnet import FFNet
from Utils.DataContainer import DataContainer

In [35]:
tf.__version__

'1.1.0'

In [5]:
mnist = input_data.read_data_sets('../Data/MNIST/', one_hot=True)

# hidden_desc = [
#     {'hidden_nodes': 500, 'activation': tf.nn.relu},
#     {'hidden_nodes': 500, 'activation': tf.nn.relu},
#     {'hidden_nodes': 500, 'activation': tf.nn.relu}
# ]

# ffnet = FFNet(784, 10, hidden_desc, True)

# ffnet.train(mnist, 100, 20, True)

# pred, pred_argmax = ffnet.predict(np.array([mnist.test.images[0]]))
# actu_argmax = np.argmax(mnist.test.labels[0])
# print('PRED', pred)
# print('ACTU', mnist.test.labels[0])
# print(pred_argmax[0], ' | ', actu_argmax)


Extracting ../Data/MNIST/train-images-idx3-ubyte.gz
Extracting ../Data/MNIST/train-labels-idx1-ubyte.gz
Extracting ../Data/MNIST/t10k-images-idx3-ubyte.gz
Extracting ../Data/MNIST/t10k-labels-idx1-ubyte.gz


In [6]:

data = DataContainer(mnist.train.images, mnist.train.labels, 3)

for batch in data.cross_validation_batches:
    print(batch['train']['inputs'], batch['train']['outputs'])
    print(batch['test']['inputs'], batch['test']['outputs'])

KeyboardInterrupt: 

[[4, 5, 6]]